# Extracción de Métricas de Calidad de Software desde SonarCloud

Este notebook está diseñado para cargar datos de estudiantes con proyectos de SonarCloud y extraer métricas de calidad de software utilizando la API de SonarCloud.

## Objetivos:
1. **Cargar datos** de estudiantes desde el archivo CSV con columnas Sonar_Ap1 y Sonar_Ap2
2. **Extraer métricas** de calidad desde SonarCloud API para cada proyecto
3. **Procesar y combinar** los datos para crear un dataset completo
4. **Exportar resultados** para análisis posteriores

## Métricas a Extraer:
- `bugs`: Errores detectados
- `vulnerabilities`: Vulnerabilidades de seguridad
- `code_smells`: Problemas de mantenibilidad
- `technical_debt`: Deuda técnica
- `duplicated_lines_density`: Densidad de líneas duplicadas
- `ncloc`: Número de líneas de código
- `complexity`: Complejidad ciclomática
- `reliability_rating`: Rating de confiabilidad
- `sqale_rating`: Rating de mantenibilidad (SQALE)
- `open_issues`: Problemas abiertos
- `coverage`: Cobertura de código
- `security_rating`: Rating de seguridad
- `security_hotspots`: Puntos críticos de seguridad
- `comment_lines_density`: Densidad de líneas de comentarios
- `cognitive_complexity`: Complejidad cognitiva

In [1]:
# Instalación de dependencias (ejecutar solo si es necesario)
import subprocess
import sys

def install_package(package):
    """Instalar paquete si no está disponible"""
    try:
        __import__(package)
        print(f"✅ {package} ya está instalado")
    except ImportError:
        print(f"🔄 Instalando {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} instalado exitosamente")

# Lista de paquetes requeridos
required_packages = ['pandas', 'requests']

print("🔧 Verificando e instalando dependencias...")
for package in required_packages:
    install_package(package)

print("✅ Todas las dependencias están listas")

🔧 Verificando e instalando dependencias...
✅ pandas ya está instalado
✅ requests ya está instalado
✅ Todas las dependencias están listas


In [2]:
# Importar librerías necesarias
import pandas as pd
import requests
import json
import base64
import time
from typing import Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas correctamente")
print("📝 Pandas version:", pd.__version__)
print("🌐 Requests disponible para API calls")

✅ Librerías importadas correctamente
📝 Pandas version: 2.3.1
🌐 Requests disponible para API calls


## 1. Configuración del Entorno

En esta sección importamos todas las librerías necesarias para el procesamiento de datos y la comunicación con la API de SonarCloud.

In [3]:
# Configuración de SonarCloud API
SONAR_TOKEN = "sqa_f5eb0e4e9b4a89c8f4e8b97f096cbc3b5dae5b09"
SONAR_BASE_URL = "https://sonarcloud.io/api"

# Configurar headers para autenticación
def get_auth_headers():
    """Crear headers de autenticación para SonarCloud API"""
    # Codificar token en base64 para autenticación básica
    auth_string = f"{SONAR_TOKEN}:"
    auth_bytes = auth_string.encode('ascii')
    auth_b64 = base64.b64encode(auth_bytes).decode('ascii')
    
    return {
        'Authorization': f'Basic {auth_b64}',
        'Content-Type': 'application/json'
    }

# Definir métricas a extraer
METRICS = [
    "bugs",                    # Errores detectados
    "vulnerabilities",         # Vulnerabilidades de seguridad  
    "code_smells",            # Problemas de mantenibilidad
    "technical_debt",         # Deuda técnica
    "duplicated_lines_density", # Densidad de líneas duplicadas
    "ncloc",                  # Número de líneas de código
    "complexity",             # Complejidad ciclomática
    "reliability_rating",     # Rating de confiabilidad
    "sqale_rating",          # Rating de mantenibilidad (SQALE)
    "open_issues",           # Problemas abiertos
    "coverage",              # Cobertura de código
    "security_rating",       # Rating de seguridad
    "security_hotspots",     # Puntos críticos de seguridad
    "comment_lines_density", # Densidad de líneas de comentarios
    "cognitive_complexity"   # Complejidad cognitiva
]

print("🔐 Configuración de autenticación preparada")
print(f"📊 {len(METRICS)} métricas configuradas para extracción")
print("✅ Headers de API configurados")

🔐 Configuración de autenticación preparada
📊 15 métricas configuradas para extracción
✅ Headers de API configurados


## 2. Configuración de la API de SonarCloud

Configuramos los parámetros necesarios para conectarnos a SonarCloud:

- **Token de autenticación**: Credenciales para acceder a la API
- **Headers de autenticación**: Configuración básica con base64 encoding
- **Métricas objetivo**: Lista de 15 métricas de calidad de software a extraer

### Métricas de Calidad Configuradas:
Las métricas se dividen en varias categorías:
- **Errores y vulnerabilidades**: `bugs`, `vulnerabilities`, `security_hotspots`
- **Mantenibilidad**: `code_smells`, `technical_debt`, `sqale_rating`
- **Complejidad**: `complexity`, `cognitive_complexity`
- **Cobertura y documentación**: `coverage`, `comment_lines_density`
- **Duplicación**: `duplicated_lines_density`
- **Tamaño**: `ncloc` (lines of code)
- **Ratings**: `reliability_rating`, `security_rating`

In [4]:
# Cargar datos de estudiantes
CSV_PATH = "Estudiantes_2023-2024.csv"

try:
    # Cargar el CSV con información de estudiantes
    df_estudiantes = pd.read_csv(CSV_PATH)
    
    print("📁 Datos cargados exitosamente")
    print(f"👥 Número de estudiantes: {len(df_estudiantes)}")
    print(f"📊 Columnas disponibles: {list(df_estudiantes.columns)}")
    
    # Mostrar información básica del dataset
    print("\n🔍 Primeras 3 filas del dataset:")
    display(df_estudiantes.head(3))
    
    # Verificar columnas de SonarCloud
    sonar_columns = [col for col in df_estudiantes.columns if 'Sonar' in col]
    print(f"\n🎯 Columnas de SonarCloud encontradas: {sonar_columns}")
    
    # Contar proyectos no vacíos
    sonar_ap1_count = df_estudiantes['Sonar_Ap1'].notna().sum()
    sonar_ap2_count = df_estudiantes['Sonar_Ap2'].notna().sum()
    
    print(f"📈 Proyectos Sonar_Ap1 disponibles: {sonar_ap1_count}")
    print(f"📈 Proyectos Sonar_Ap2 disponibles: {sonar_ap2_count}")
    print(f"📈 Total de proyectos a procesar: {sonar_ap1_count + sonar_ap2_count}")

except FileNotFoundError:
    print("❌ Error: No se encontró el archivo CSV")
    print(f"📍 Buscando en: {CSV_PATH}")
except Exception as e:
    print(f"❌ Error al cargar datos: {e}")

📁 Datos cargados exitosamente
👥 Número de estudiantes: 60
📊 Columnas disponibles: ['Id', 'Semestre', 'Estudiante', 'Sexo', 'Email', 'Original_Repo_Ap1', 'Original_Repo_Ap2', 'Sonar_Ap1', 'Sonar_Ap2', 'Sonar_Repo_Ap1', 'Sonar_Repo_Ap2']

🔍 Primeras 3 filas del dataset:


,Id,Semestre,Estudiante,Sexo,Email,Original_Repo_Ap1,Original_Repo_Ap2,Sonar_Ap1,Sonar_Ap2,Sonar_Repo_Ap1,Sonar_Repo_Ap2
0,1,2024-01,Aaron Eliezer Hernández García,1,atminifg655@gmail.com,https://github.com/aaron-developer25/SwiftPay.git,https://github.com/aaron-developer25/DealerPOS...,TesisEnel_SwiftPay-Aaron-Ap1,TesisEnel_DealerPOS-Aaron-ap2,https://github.com/TesisEnel/SwiftPay-Aaron-Ap1,https://github.com/TesisEnel/DealerPOS-Aaron-ap2
1,2,2023-03,Abraham El Hage Jreij,1,abrahamelhage2003@gmail.com,https://github.com/JPichardo2003/AguaMariaSolu...,https://github.com/A-EHJ/Final_Project_Ap2.git,TesisEnel_AguaMariaSolution-JulioPichardo-ap1,TesisEnel_Final_Project-Abraham-ap2,https://github.com/TesisEnel/AguaMariaSolution...,https://github.com/TesisEnel/Final_Project-Abr...
2,3,2024-02,Adiel Luis García Rosa,1,adiel.garcia0422@gmail.com,https://github.com/SamyJp23/PeakPerformance.git,https://github.com/Adiel040/GymProApp.git,TesisEnel_PeakPerformance-samuelAntonio-ap1,TesisEnel_GymProApp-AdielGarcia-Ap2,https://github.com/TesisEnel/PeakPerformance-s...,https://github.com/TesisEnel/GymProApp-AdielGa...



🎯 Columnas de SonarCloud encontradas: ['Sonar_Ap1', 'Sonar_Ap2', 'Sonar_Repo_Ap1', 'Sonar_Repo_Ap2']
📈 Proyectos Sonar_Ap1 disponibles: 60
📈 Proyectos Sonar_Ap2 disponibles: 60
📈 Total de proyectos a procesar: 120


## 3. Carga de Datos de Estudiantes

Cargamos el archivo CSV que contiene la información de los estudiantes, incluyendo las columnas `Sonar_Ap1` y `Sonar_Ap2` que contienen las claves de los proyectos en SonarCloud.

### Estructura esperada del CSV:
- **ID**: Identificador único del estudiante
- **Nombre**: Nombre del estudiante  
- **Sonar_Ap1**: Clave del proyecto de la Aplicación 1 en SonarCloud
- **Sonar_Ap2**: Clave del proyecto de la Aplicación 2 en SonarCloud
- **Otras columnas**: Información adicional del estudiante

In [ ]:
# Extraer project keys de SonarCloud
def extract_project_keys(df):
    """
    Extraer todas las claves de proyecto de SonarCloud del DataFrame
    """
    project_keys = []
    
    for index, row in df.iterrows():
        student_id = row.get('ID', f'Student_{index}')
        nombre = row.get('Nombre', 'Unknown')
        
        # Procesar Sonar_Ap1
        if pd.notna(row['Sonar_Ap1']) and row['Sonar_Ap1'].strip():
            project_keys.append({
                'student_id': student_id,
                'nombre': nombre,
                'project_key': row['Sonar_Ap1'].strip(),
                'assignment': 'AP1',
                'row_index': index
            })
        
        # Procesar Sonar_Ap2
        if pd.notna(row['Sonar_Ap2']) and row['Sonar_Ap2'].strip():
            project_keys.append({
                'student_id': student_id,
                'nombre': nombre,
                'project_key': row['Sonar_Ap2'].strip(),
                'assignment': 'AP2',
                'row_index': index
            })
    
    return project_keys

# Extraer project keys
project_list = extract_project_keys(df_estudiantes)

print(f"🔑 Total de project keys extraídos: {len(project_list)}")

# Mostrar estadísticas por assignment
ap1_count = len([p for p in project_list if p['assignment'] == 'AP1'])
ap2_count = len([p for p in project_list if p['assignment'] == 'AP2'])

print(f"📊 Proyectos AP1: {ap1_count}")
print(f"📊 Proyectos AP2: {ap2_count}")

# Mostrar algunos ejemplos
print("\n🔍 Primeros 5 project keys:")
for i, project in enumerate(project_list[:5]):
    print(f"  {i+1}. {project['nombre']} - {project['assignment']}: {project['project_key']}")

# Crear DataFrame con los project keys
df_projects = pd.DataFrame(project_list)
print(f"\n✅ DataFrame de proyectos creado con {len(df_projects)} filas")

## 4. Extracción de Claves de Proyecto

Procesamos el DataFrame de estudiantes para extraer todas las claves de proyecto de SonarCloud disponibles. Cada estudiante puede tener hasta 2 proyectos (AP1 y AP2).

### Proceso de extracción:
1. **Iteración por estudiante**: Revisamos cada fila del DataFrame
2. **Validación de datos**: Verificamos que las celdas no estén vacías
3. **Estructuración**: Creamos una lista con la información del proyecto y estudiante
4. **Categorización**: Separamos por assignment (AP1 vs AP2)

El resultado es una lista estructurada que facilita el procesamiento posterior con la API.

In [ ]:
# Funciones para interactuar con SonarCloud API
def fetch_project_metrics(project_key: str, metrics_list: List[str]) -> Dict:
    """
    Obtener métricas de un proyecto específico desde SonarCloud
    """
    url = f"{SONAR_BASE_URL}/measures/component"
    
    # Parámetros para la API
    params = {
        'component': project_key,
        'metricKeys': ','.join(metrics_list)
    }
    
    try:
        # Realizar petición a la API
        response = requests.get(url, params=params, headers=get_auth_headers(), timeout=30)
        
        if response.status_code == 200:
            data = response.json()
            
            # Extraer métricas del response
            metrics_dict = {'project_key': project_key, 'status': 'success'}
            
            if 'component' in data and 'measures' in data['component']:
                for measure in data['component']['measures']:
                    metric_key = measure['metric']
                    metric_value = measure.get('value', None)
                    
                    # Convertir valores numéricos
                    if metric_value is not None:
                        try:
                            # Intentar convertir a float
                            if '.' in str(metric_value) or metric_key in ['coverage', 'duplicated_lines_density', 'comment_lines_density']:
                                metrics_dict[metric_key] = float(metric_value)
                            else:
                                metrics_dict[metric_key] = int(metric_value)
                        except ValueError:
                            # Mantener como string si no se puede convertir
                            metrics_dict[metric_key] = metric_value
                    else:
                        metrics_dict[metric_key] = None
            
            # Agregar métricas faltantes como None
            for metric in metrics_list:
                if metric not in metrics_dict:
                    metrics_dict[metric] = None
                    
            return metrics_dict
            
        else:
            print(f"⚠️  Error {response.status_code} para proyecto {project_key}")
            return {'project_key': project_key, 'status': 'error', 'error_code': response.status_code}
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Error de conexión para proyecto {project_key}: {e}")
        return {'project_key': project_key, 'status': 'connection_error', 'error': str(e)}
    except Exception as e:
        print(f"❌ Error inesperado para proyecto {project_key}: {e}")
        return {'project_key': project_key, 'status': 'unexpected_error', 'error': str(e)}

def batch_fetch_metrics(project_list: List[Dict], batch_size: int = 5, delay: float = 1.0) -> List[Dict]:
    """
    Obtener métricas para múltiples proyectos con control de rate limiting
    """
    results = []
    total_projects = len(project_list)
    
    print(f"🚀 Iniciando extracción de métricas para {total_projects} proyectos")
    print(f"⚙️  Batch size: {batch_size}, Delay: {delay}s")
    
    for i in range(0, total_projects, batch_size):
        batch = project_list[i:i + batch_size]
        batch_num = (i // batch_size) + 1
        total_batches = (total_projects + batch_size - 1) // batch_size
        
        print(f"\n📦 Procesando batch {batch_num}/{total_batches} ({len(batch)} proyectos)")
        
        for project in batch:
            project_key = project['project_key']
            print(f"  🔄 Extrayendo métricas de: {project_key}")
            
            # Obtener métricas
            metrics = fetch_project_metrics(project_key, METRICS)
            
            # Combinar información del proyecto con las métricas
            result = {**project, **metrics}
            results.append(result)
            
            # Status indicator
            if metrics['status'] == 'success':
                print(f"    ✅ Éxito")
            else:
                print(f"    ❌ Error: {metrics.get('status', 'unknown')}")
        
        # Delay entre batches para evitar rate limiting
        if i + batch_size < total_projects:
            print(f"  ⏳ Esperando {delay}s antes del siguiente batch...")
            time.sleep(delay)
    
    print(f"\n🎉 Extracción completada: {len(results)} proyectos procesados")
    return results

print("🛠️  Funciones de API configuradas")
print("📡 Listo para extraer métricas de SonarCloud")

## 5. Funciones de Interacción con la API

Definimos las funciones principales para interactuar con la API de SonarCloud de manera eficiente y robusta.

### Funciones implementadas:

#### `fetch_project_metrics(project_key, metrics_list)`
- **Propósito**: Obtener métricas de un proyecto específico
- **Endpoint**: `/api/measures/component`
- **Manejo de errores**: Captura errores de conexión, timeouts y respuestas inválidas
- **Conversión de tipos**: Convierte automáticamente valores numéricos

#### `batch_fetch_metrics(project_list, batch_size, delay)`
- **Propósito**: Procesar múltiples proyectos con control de rate limiting
- **Rate limiting**: Implementa delays entre batches para evitar sobrecargar la API
- **Monitoreo**: Proporciona feedback en tiempo real del progreso
- **Tolerancia a fallos**: Continúa procesando aunque fallen proyectos individuales

In [ ]:
# Ejecutar extracción de métricas
print("🚀 Iniciando proceso de extracción de métricas...")
print(f"📊 Total de proyectos a procesar: {len(project_list)}")

# Configurar parámetros de extracción
BATCH_SIZE = 3  # Reducir para evitar rate limiting
DELAY_BETWEEN_BATCHES = 2.0  # Segundos de espera entre batches

# Ejecutar extracción
metrics_results = batch_fetch_metrics(
    project_list=project_list,
    batch_size=BATCH_SIZE,
    delay=DELAY_BETWEEN_BATCHES
)

print("\n📈 Resultados de la extracción:")
print(f"✅ Total de proyectos procesados: {len(metrics_results)}")

# Analizar resultados
successful_extractions = [r for r in metrics_results if r.get('status') == 'success']
failed_extractions = [r for r in metrics_results if r.get('status') != 'success']

print(f"✅ Extracciones exitosas: {len(successful_extractions)}")
print(f"❌ Extracciones fallidas: {len(failed_extractions)}")

if failed_extractions:
    print("\n⚠️  Proyectos con errores:")
    for failed in failed_extractions[:5]:  # Mostrar solo los primeros 5
        print(f"  - {failed['project_key']}: {failed.get('status', 'unknown error')}")

# Mostrar ejemplo de métricas extraídas
if successful_extractions:
    print("\n🔍 Ejemplo de métricas extraídas (primer proyecto exitoso):")
    example = successful_extractions[0]
    print(f"Proyecto: {example['project_key']}")
    print(f"Estudiante: {example['nombre']}")
    print(f"Assignment: {example['assignment']}")
    
    # Mostrar algunas métricas clave
    key_metrics = ['bugs', 'vulnerabilities', 'code_smells', 'ncloc', 'coverage']
    for metric in key_metrics:
        value = example.get(metric, 'N/A')
        print(f"  {metric}: {value}")
        
print("\n✅ Extracción de métricas completada")

## 6. Ejecución de la Extracción de Métricas

Ejecutamos el proceso principal de extracción de métricas. Esta es la parte más crítica del notebook donde se realiza la comunicación con SonarCloud.

### Parámetros de configuración:
- **BATCH_SIZE**: Número de proyectos a procesar por lote (3 proyectos)
- **DELAY_BETWEEN_BATCHES**: Tiempo de espera entre lotes (2 segundos)

### Monitoreo del proceso:
- ✅ **Extracciones exitosas**: Proyectos procesados correctamente
- ❌ **Extracciones fallidas**: Proyectos con errores (ej: proyecto no encontrado, permisos)
- 📊 **Progreso en tiempo real**: Indicadores de estado por cada proyecto

> **Nota**: El proceso puede tomar varios minutos dependiendo del número de proyectos. Los delays son necesarios para respetar los límites de la API de SonarCloud.

In [ ]:
# Procesar y estructurar los datos extraídos
print("🔄 Procesando datos extraídos...")

# Crear DataFrame con las métricas extraídas
df_metrics = pd.DataFrame(successful_extractions)

if len(df_metrics) > 0:
    print(f"📊 DataFrame de métricas creado con {len(df_metrics)} filas")
    
    # Mostrar información del DataFrame
    print(f"📋 Columnas disponibles: {len(df_metrics.columns)}")
    
    # Separar columnas de información del proyecto vs métricas
    info_columns = ['student_id', 'nombre', 'project_key', 'assignment', 'row_index', 'status']
    metric_columns = [col for col in df_metrics.columns if col not in info_columns]
    
    print(f"📈 Métricas extraídas: {len(metric_columns)}")
    print(f"ℹ️  Columnas de información: {len(info_columns)}")
    
    # Mostrar estadísticas básicas de las métricas numéricas
    numeric_metrics = df_metrics[metric_columns].select_dtypes(include=['number'])
    
    if len(numeric_metrics.columns) > 0:
        print(f"\n📊 Estadísticas descriptivas de métricas numéricas:")
        print(f"Métricas numéricas encontradas: {list(numeric_metrics.columns)}")
        
        # Mostrar estadísticas básicas
        stats = numeric_metrics.describe()
        display(stats.round(2))
        
        # Verificar valores nulos
        null_counts = df_metrics[metric_columns].isnull().sum()
        print(f"\n❓ Valores nulos por métrica:")
        for metric, null_count in null_counts.items():
            if null_count > 0:
                print(f"  {metric}: {null_count}/{len(df_metrics)} ({null_count/len(df_metrics)*100:.1f}%)")
    
    # Mostrar distribución por assignment
    assignment_dist = df_metrics['assignment'].value_counts()
    print(f"\n📊 Distribución por assignment:")
    for assignment, count in assignment_dist.items():
        print(f"  {assignment}: {count} proyectos")
    
    # Mostrar primeras filas del DataFrame
    print(f"\n🔍 Primeras 3 filas del dataset de métricas:")
    display(df_metrics[info_columns + metric_columns[:5]].head(3))
    
else:
    print("❌ No se pudieron procesar las métricas extraídas")
    df_metrics = pd.DataFrame()

print("✅ Procesamiento de datos completado")

## 7. Procesamiento y Análisis de Datos Extraídos

Convertimos los resultados de la extracción en un DataFrame estructurado y realizamos un análisis inicial de la calidad de los datos.

### Análisis incluido:
- **📊 Estadísticas descriptivas**: Media, mediana, min, max para métricas numéricas
- **❓ Análisis de valores nulos**: Identificación de métricas faltantes
- **📈 Distribución por assignment**: Conteo de proyectos AP1 vs AP2
- **🔍 Vista previa**: Primeras filas del dataset para validación

### Separación de columnas:
- **Información del proyecto**: student_id, nombre, project_key, assignment
- **Métricas de calidad**: bugs, vulnerabilities, code_smells, etc.

Este análisis nos permite identificar patrones y posibles problemas en los datos antes de continuar.

In [ ]:
# Combinar datos de estudiantes con métricas extraídas
def merge_student_data_with_metrics(df_estudiantes, df_metrics):
    """
    Combinar datos originales de estudiantes con las métricas extraídas
    """
    print("🔗 Iniciando proceso de combinación de datos...")
    
    # Crear copias para no modificar los originales
    df_students_copy = df_estudiantes.copy()
    df_metrics_copy = df_metrics.copy()
    
    # Agregar sufijos a las métricas según el assignment
    merged_data = []
    
    for _, student_row in df_students_copy.iterrows():
        student_data = student_row.to_dict()
        
        # Buscar métricas para AP1
        ap1_metrics = df_metrics_copy[
            (df_metrics_copy['row_index'] == student_row.name) & 
            (df_metrics_copy['assignment'] == 'AP1')
        ]
        
        if len(ap1_metrics) > 0:
            ap1_row = ap1_metrics.iloc[0]
            for metric in METRICS:
                student_data[f'{metric}_AP1'] = ap1_row.get(metric, None)
        else:
            # Agregar valores nulos si no hay métricas AP1
            for metric in METRICS:
                student_data[f'{metric}_AP1'] = None
        
        # Buscar métricas para AP2
        ap2_metrics = df_metrics_copy[
            (df_metrics_copy['row_index'] == student_row.name) & 
            (df_metrics_copy['assignment'] == 'AP2')
        ]
        
        if len(ap2_metrics) > 0:
            ap2_row = ap2_metrics.iloc[0]
            for metric in METRICS:
                student_data[f'{metric}_AP2'] = ap2_row.get(metric, None)
        else:
            # Agregar valores nulos si no hay métricas AP2
            for metric in METRICS:
                student_data[f'{metric}_AP2'] = None
        
        merged_data.append(student_data)
    
    return pd.DataFrame(merged_data)

# Ejecutar combinación de datos
if len(df_metrics) > 0:
    df_combined = merge_student_data_with_metrics(df_estudiantes, df_metrics)
    
    print(f"✅ Datos combinados exitosamente")
    print(f"👥 Estudiantes en dataset combinado: {len(df_combined)}")
    print(f"📊 Total de columnas: {len(df_combined.columns)}")
    
    # Contar columnas de métricas añadidas
    metric_ap1_cols = [col for col in df_combined.columns if col.endswith('_AP1')]
    metric_ap2_cols = [col for col in df_combined.columns if col.endswith('_AP2')]
    
    print(f"📈 Columnas de métricas AP1: {len(metric_ap1_cols)}")
    print(f"📈 Columnas de métricas AP2: {len(metric_ap2_cols)}")
    
    # Verificar cobertura de datos
    ap1_projects_with_metrics = df_combined[metric_ap1_cols].notna().any(axis=1).sum()
    ap2_projects_with_metrics = df_combined[metric_ap2_cols].notna().any(axis=1).sum()
    
    print(f"\n📊 Cobertura de métricas:")
    print(f"  Estudiantes con métricas AP1: {ap1_projects_with_metrics}")
    print(f"  Estudiantes con métricas AP2: {ap2_projects_with_metrics}")
    
    # Mostrar algunas columnas del dataset combinado
    sample_columns = ['Nombre', 'Sonar_Ap1', 'Sonar_Ap2', 'bugs_AP1', 'bugs_AP2', 'ncloc_AP1', 'ncloc_AP2']
    available_sample_cols = [col for col in sample_columns if col in df_combined.columns]
    
    if available_sample_cols:
        print(f"\n🔍 Vista previa del dataset combinado:")
        display(df_combined[available_sample_cols].head(3))
    
else:
    print("❌ No se pueden combinar datos sin métricas extraídas")
    df_combined = df_estudiantes.copy()

print("✅ Combinación de datos completada")

## 8. Combinación de Datos de Estudiantes con Métricas

Integramos las métricas extraídas con la información original de los estudiantes para crear un dataset completo y análisis-ready.

### Proceso de combinación:
1. **Mantenimiento de estructura original**: Preservamos todas las columnas del CSV original
2. **Adición de métricas por assignment**: Cada métrica se agrega con sufijos `_AP1` y `_AP2`
3. **Manejo de datos faltantes**: Asignamos `None` cuando no hay métricas disponibles
4. **Validación de cobertura**: Calculamos estadísticas de completitud de datos

### Estructura del dataset final:
- **Columnas originales**: ID, Nombre, Sonar_Ap1, Sonar_Ap2, etc.
- **Métricas AP1**: bugs_AP1, vulnerabilities_AP1, ncloc_AP1, etc.
- **Métricas AP2**: bugs_AP2, vulnerabilities_AP2, ncloc_AP2, etc.

### Ejemplo de nuevas columnas añadidas:
```
bugs_AP1, bugs_AP2
vulnerabilities_AP1, vulnerabilities_AP2  
code_smells_AP1, code_smells_AP2
ncloc_AP1, ncloc_AP2
coverage_AP1, coverage_AP2
... (total: 30 nuevas columnas)
```

In [ ]:
# Exportar resultados y generar reportes
from datetime import datetime

# Crear timestamp para los archivos
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

print("💾 Preparando exportación de resultados...")

# 1. Exportar dataset combinado completo
output_file_combined = f"estudiantes_con_metricas_sonarcloud_{timestamp}.csv"

try:
    df_combined.to_csv(output_file_combined, index=False, encoding='utf-8')
    print(f"✅ Dataset combinado exportado: {output_file_combined}")
    print(f"📊 Filas exportadas: {len(df_combined)}")
    print(f"📋 Columnas exportadas: {len(df_combined.columns)}")
except Exception as e:
    print(f"❌ Error al exportar dataset combinado: {e}")

# 2. Exportar solo las métricas extraídas
if len(df_metrics) > 0:
    output_file_metrics = f"metricas_sonarcloud_raw_{timestamp}.csv"
    try:
        df_metrics.to_csv(output_file_metrics, index=False, encoding='utf-8')
        print(f"✅ Métricas raw exportadas: {output_file_metrics}")
        print(f"📊 Proyectos exportados: {len(df_metrics)}")
    except Exception as e:
        print(f"❌ Error al exportar métricas raw: {e}")

# 3. Generar reporte de resumen
print(f"\n📋 REPORTE DE EXTRACCIÓN DE MÉTRICAS")
print(f"=" * 50)
print(f"🕐 Fecha y hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"👥 Total de estudiantes: {len(df_estudiantes)}")
print(f"🎯 Proyectos identificados: {len(project_list)}")
print(f"✅ Métricas extraídas exitosamente: {len(successful_extractions)}")
print(f"❌ Extracciones fallidas: {len(failed_extractions)}")
print(f"📊 Métricas por proyecto: {len(METRICS)}")

if len(df_combined) > 0:
    print(f"\n📈 ESTADÍSTICAS DEL DATASET FINAL:")
    
    # Contar estudiantes con datos completos
    students_with_ap1 = (df_combined[[col for col in df_combined.columns if col.endswith('_AP1')]].notna().any(axis=1)).sum()
    students_with_ap2 = (df_combined[[col for col in df_combined.columns if col.endswith('_AP2')]].notna().any(axis=1)).sum()
    students_with_both = (
        (df_combined[[col for col in df_combined.columns if col.endswith('_AP1')]].notna().any(axis=1)) &
        (df_combined[[col for col in df_combined.columns if col.endswith('_AP2')]].notna().any(axis=1))
    ).sum()
    
    print(f"  📊 Estudiantes con métricas AP1: {students_with_ap1}")
    print(f"  📊 Estudiantes con métricas AP2: {students_with_ap2}")
    print(f"  📊 Estudiantes con ambas métricas: {students_with_both}")
    
    # Mostrar cobertura por métrica
    print(f"\n📊 COBERTURA POR MÉTRICA (% de proyectos con datos):")
    for metric in METRICS[:5]:  # Mostrar las primeras 5 métricas
        ap1_coverage = df_combined[f'{metric}_AP1'].notna().sum() / len(df_combined) * 100
        ap2_coverage = df_combined[f'{metric}_AP2'].notna().sum() / len(df_combined) * 100
        print(f"  {metric}: AP1={ap1_coverage:.1f}%, AP2={ap2_coverage:.1f}%")

# 4. Guardar configuración utilizada
config_info = {
    'timestamp': timestamp,
    'total_students': len(df_estudiantes),
    'projects_found': len(project_list),
    'metrics_extracted': len(successful_extractions),
    'extraction_failures': len(failed_extractions),
    'metrics_configured': METRICS,
    'batch_size': BATCH_SIZE,
    'delay_between_batches': DELAY_BETWEEN_BATCHES
}

config_file = f"extraction_config_{timestamp}.json"
try:
    with open(config_file, 'w', encoding='utf-8') as f:
        json.dump(config_info, f, indent=2, ensure_ascii=False)
    print(f"\n✅ Configuración guardada: {config_file}")
except Exception as e:
    print(f"❌ Error al guardar configuración: {e}")

print(f"\n🎉 EXTRACCIÓN COMPLETADA EXITOSAMENTE!")
print(f"📁 Archivos generados disponibles en el directorio de trabajo")
print(f"📊 Listo para análisis posterior de métricas de calidad de software")

## 9. Exportación de Resultados y Generación de Reportes

Finalizamos el proceso exportando los datos procesados y generando reportes comprensivos del proceso de extracción.

### Archivos generados:

#### 1. Dataset Combinado
- **Archivo**: `estudiantes_con_metricas_sonarcloud_{timestamp}.csv`
- **Contenido**: Datos originales de estudiantes + 30 columnas de métricas
- **Uso**: Análisis estadístico, visualizaciones, machine learning

#### 2. Métricas Raw
- **Archivo**: `metricas_sonarcloud_raw_{timestamp}.csv`
- **Contenido**: Métricas extraídas en formato largo (una fila por proyecto)
- **Uso**: Análisis detallado por proyecto, debugging

#### 3. Configuración de Extracción
- **Archivo**: `extraction_config_{timestamp}.json`
- **Contenido**: Parámetros utilizados, métricas configuradas, estadísticas
- **Uso**: Reproducibilidad, documentación del proceso

### Reporte de extracción:
- **📊 Estadísticas generales**: Total de estudiantes, proyectos procesados
- **✅ Tasa de éxito**: Extracciones exitosas vs fallidas
- **📈 Cobertura de datos**: Porcentaje de estudiantes con métricas completas
- **📋 Métricas por categoría**: Distribución de completitud por métrica

> **Importante**: Todos los archivos incluyen un timestamp para evitar sobrescribir resultados anteriores y mantener un historial de extracciones.

## ✅ Proceso Completado

### Resumen del flujo ejecutado:
1. ✅ **Configuración**: Librerías y API de SonarCloud configuradas
2. ✅ **Carga de datos**: CSV de estudiantes procesado
3. ✅ **Extracción de claves**: Project keys identificados y estructurados
4. ✅ **Conexión API**: Funciones de extracción implementadas
5. ✅ **Extracción masiva**: Métricas obtenidas con rate limiting
6. ✅ **Procesamiento**: Datos estructurados y analizados
7. ✅ **Combinación**: Dataset final con métricas integradas
8. ✅ **Exportación**: Archivos generados para análisis posterior

### Próximos pasos sugeridos:
- **📊 Análisis Exploratorio**: Usar el dataset combinado para análisis estadístico
- **📈 Visualizaciones**: Crear gráficos de distribución de métricas por assignment
- **🔍 Análisis Comparativo**: Comparar métricas entre AP1 y AP2
- **🎯 Análisis de Calidad**: Identificar patrones en la calidad del código
- **📋 Reportes**: Generar reportes de progreso para estudiantes

### Archivos disponibles para análisis:
Los archivos CSV generados están listos para ser importados en herramientas de análisis como:
- **Python**: pandas, matplotlib, seaborn
- **R**: ggplot2, dplyr
- **Excel/Google Sheets**: Para análisis básico
- **Tableau/Power BI**: Para dashboards interactivos